# Gold Layer Orchestration

In [0]:
# ==========================================
# GOLD LAYER ORCHESTRATION 
# ==========================================

import time
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

notebooks = [
    "./gold_dim_customers",
    "./gold_dim_products",
    "./gold_fact_sales"
      ]

results = []

print("Starting Gold orchestration...\n")

for notebook in notebooks:
    start = time.time()
    print(f"Running → {notebook}")

    try:
        output = dbutils.notebook.run(notebook, 600)
        status = "SUCCESS"
    except Exception as e:
        output = str(e)
        status = "FAILED"

    duration = round(time.time() - start, 2)

    print(f"{notebook} → {status} ({duration}s)")
    print(f"Output/Error → {str(output)[:500]}\n")

    results.append((str(notebook), str(status), float(duration), str(output)))

# ---- explicit schema avoids inference error ----
schema = StructType([
    StructField("notebook", StringType(), True),
    StructField("status", StringType(), True),
    StructField("duration_sec", DoubleType(), True),
    StructField("output", StringType(), True)
])

result_df = spark.createDataFrame(results, schema)

display(result_df)

print("Gold orchestration finished.")